# Action Potential and Spike Train Analysis

## Introduction
This Jupyter Notebook provides a comprehensive workflow for analyzing action potential properties and spike train dynamics from electrophysiological data. The analysis includes detecting action potentials, calculating their properties (e.g., amplitude, half-width), and examining spike train dynamics such as firing rate and interspike interval (ISI) distributions.

## Import Libraries
Import the necessary Python libraries for data manipulation, analysis, and visualization.

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import os

## Load Data
Define a function to load electrophysiology data from a CSV file. This function will handle errors related to missing files or incorrect data formats.

In [ ]:
def load_data(file_path):
    """
    Load electrophysiology data from a CSV file.
    
    Args:
        file_path (str): Path to the input data file.
    
    Returns:
        data (DataFrame): Loaded data as a Pandas DataFrame.
    
    Raises:
        FileNotFoundError: If the file is not found at the specified path.
        ValueError: If the data format is incorrect or missing columns.
    """
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file '{file_path}' does not exist.")
    
    data = pd.read_csv(file_path)
    if 'Time' not in data.columns or 'Voltage' not in data.columns:
        raise ValueError("The input data must contain 'Time' and 'Voltage' columns.")
    
    return data

# Load the data
file_path = '../data/action_potential_data.csv'  # Update this path as needed
try:
    data = load_data(file_path)
    display(data.head())  # Show the first few rows of the data
except Exception as e:
    print(f"Error loading data: {e}")

## Analyse Action Potentials
Define a function to detect action potentials and calculate their properties, such as amplitude and half-width.

In [ ]:
def analyze_action_potentials(data, threshold=0):
    """
    Analyze action potential properties in electrophysiology data.

    Args:
        data (DataFrame): Electrophysiology data containing time and voltage columns.
        threshold (float): Voltage threshold for peak detection.

    Returns:
        ap_properties (DataFrame): DataFrame containing detected AP properties (e.g., amplitude, half-width).
    """
    time = data['Time'].values
    voltage = data['Voltage'].values
    
    # Detect action potentials (APs) using peak detection
    peaks, _ = find_peaks(voltage, height=threshold)

    # Calculate action potential properties
    amplitudes = voltage[peaks]
    half_widths = np.diff(peaks) * (time[1] - time[0])  # Rough estimate based on sample rate

    ap_properties = pd.DataFrame({
        'AP_Index': peaks,
        'Amplitude': amplitudes,
        'Half_Width': half_widths
    })

    return ap_properties

# Perform AP analysis
threshold = 0  # Set the threshold for AP detection
try:
    ap_properties = analyze_action_potentials(data, threshold=threshold)
    display(ap_properties.head())  # Show the first few rows of the AP properties
except Exception as e:
    print(f"Error in AP analysis: {e}")

## Analyze Spike Train Dynamics
Define a function to analyze spike train dynamics, including firing rate histograms and interspike interval (ISI) distributions.

In [ ]:
def analyze_spike_train(ap_properties, bin_size=0.01):
    """
    Analyze spike train dynamics such as firing rate and interspike interval (ISI) distribution.

    Args:
        ap_properties (DataFrame): DataFrame containing detected action potentials.
        bin_size (float): Bin size for calculating firing rate histogram (in seconds).

    Returns:
        firing_rate_hist (ndarray): Firing rate histogram.
        isi_distribution (ndarray): Interspike interval distribution.
    """
    spike_times = ap_properties['AP_Index'].values * (data['Time'][1] - data['Time'][0])  # Convert index to time
    
    # Calculate ISI (Interspike Interval)
    isi = np.diff(spike_times)

    # Calculate Firing Rate Histogram
    firing_rate_hist, bin_edges = np.histogram(spike_times, bins=np.arange(0, spike_times[-1] + bin_size, bin_size))
    
    return firing_rate_hist, isi

# Analyze spike train
bin_size = 0.01  # Bin size for firing rate histogram
try:
    firing_rate_hist, isi = analyze_spike_train(ap_properties, bin_size=bin_size)
except Exception as e:
    print(f"Error in spike train analysis: {e}")

## Plot Results
Define a function to visualize the action potential detection, firing rate histogram, and ISI distribution.

In [ ]:
def plot_results(data, ap_properties, firing_rate_hist, isi):
    """
    Plot action potential properties, firing rate histogram, and ISI distribution.

    Args:
        data (DataFrame): Electrophysiology data.
        ap_properties (DataFrame): Detected action potential properties.
        firing_rate_hist (ndarray): Firing rate histogram.
        isi (ndarray): Interspike interval distribution.
    """
    # Plot raw data with detected APs
    plt.figure(figsize=(10, 4))
    plt.plot(data['Time'], data['Voltage'], label='Voltage Trace')
    plt.scatter(data['Time'][ap_properties['AP_Index']], ap_properties['Amplitude'], color='red', label='Detected APs')
    plt.xlabel('Time (s)')
    plt.ylabel('Voltage (mV)')
    plt.title('Action Potential Detection')
    plt.legend()
    plt.show()

    # Plot firing rate histogram
    plt.figure(figsize=(10, 4))
    plt.hist(firing_rate_hist, bins=50, color='blue', alpha=0.7)
    plt.xlabel('Firing Rate (Hz)')
    plt.ylabel('Count')
    plt.title('Firing Rate Histogram')
    plt.show()

    # Plot ISI distribution
    plt.figure(figsize=(10, 4))
    plt.hist(isi, bins=50, color='green', alpha=0.7)
    plt.xlabel('Interspike Interval (s)')
    plt.ylabel('Count')
    plt.title('Interspike Interval (ISI) Distribution')
    plt.show()

# Plot the results
try:
    plot_results(data, ap_properties, firing_rate_hist, isi)
except Exception as e:
    print(f"Error in plotting: {e}")

## Conclusion

This notebook provides a comprehensive workflow for analyzing action potential properties and spike train dynamics from electrophysiological data. Adjust parameters such as the voltage threshold and bin size to explore different scenarios.